<a href="https://colab.research.google.com/github/RonnieRatwani/DistanceTracking/blob/main/Distancetracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join(DATA_DIR, 'models')
for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)


In [2]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2397, done.
remote: Counting objects: 100% (2397/2397), done.
remote: Compressing objects: 100% (1998/1998), done.
remote: Total 2397 (delta 571), reused 1384 (delta 372), pack-reused 0
Receiving objects: 100% (2397/2397), 30.77 MiB | 37.83 MiB/s, done.
Resolving deltas: 100% (571/571), done.


The code below is used to download the object detection model checkpoint file, as well as the labels file (.pbtxt) which contains a list of strings used to add the correct label to each detection (e.g. person).

In [3]:
import tarfile
import urllib.request

# Download and extract model
MODEL_DATE = '20200711'
MODEL_NAME = 'faster_rcnn_resnet101_v1_640x640_coco17_tpu-8'
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME #+ MODEL_DATE + '/'
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
    print('Downloading model. This may take a while... ', end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_DIR)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Done')

# Download labels file
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_DOWNLOAD_BASE = \
    'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))
if not os.path.exists(PATH_TO_LABELS):
    print('Downloading label file... ', end='')
    urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE + LABEL_FILENAME, PATH_TO_LABELS)
    print('Done')

Load the Downloaded model

In [4]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml

Selecting previously unselected package python-bs4.
(Reading database ... 146456 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [5]:
import os
os.chdir('models/research')

In [6]:
!protoc object_detection/protos/*.proto --python_out=.

In [7]:
import sys
sys.path.append('/content/models/research/slim')

In [8]:
!pip install tf_slim
!pip install tf-models-official

     |████████████████████████████████| 358kB 14.1MB/s 
     |████████████████████████████████| 1.1MB 16.3MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 174kB 44.8MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 1.2MB 47.4MB/s 
     |████████████████████████████████| 645kB 54.9MB/s 
     |████████████████████████████████| 37.6MB 89kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=bbfc595d19642246a46bba0e44f83aed11fd1d193cb61122da99cae9d3352365
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20072 sha256=198bb67beef491ac4894edf9f5748ce219465ccf87e0865a07b5b49668727ff9
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built seqeval py

In [9]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

## Load label map data (for plotting)
Label maps correspond index numbers to category names, so that when our convolution network
predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility
functions, but anything that returns a dictionary mapping integers to appropriate string labels
would be fine.



In [10]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,use_display_name=True)

In [11]:
category_index = {1: {'id': 1, 'name': 'person'},3: {'id': 3, 'name': 'car'}}

In [14]:
import numpy as np
import cv2
import time
from IPython.display import Image
from matplotlib import pyplot as plt


cap = cv2.VideoCapture("/content/data/output_inbuilt_trimnone.avi")
img = []

Dis = 4.2
height = 1.7

while True:
    # Read frame from camera
    ret, image_np = cap.read()
    if ret == False:
        break
#     image_np = cv2.imread("capture1.jpg")
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    start_time= time.time()
    detections, predictions_dict, shapes = detect_fn(input_tensor)
    end_time = time.time()
    
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    print(end_time-start_time)
    fps = round((1/(end_time-start_time)),2)
    fps = str(fps)
    
#     viz_utils.visualize_boxes_and_labels_on_image_array(
#           image_np_with_detections,
#           detections['detection_boxes'][0].numpy(),
#           (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
#           detections['detection_scores'][0].numpy(),
#           category_index,
#           use_normalized_coordinates=True,
#           max_boxes_to_draw=200,
#           min_score_thresh=.30,
#           skip_boxes=True,
#         skip_scores=True,
#         skip_labels = True,
#           agnostic_mode=False)
    
    box ={}
    number = 0
    for i in range(0, len(detections['detection_classes'][0].numpy())):
        if (detections['detection_classes'][0][i] == 0 or detections['detection_classes'][0][i] == 2) and detections['detection_scores'][0][i]>=.30:
            number +=1
            box[number]= detections['detection_boxes'][0][i].numpy()
    im_height, im_width,_ = image_np.shape
    file1 = open("/content/data/focal.txt","r+")
    focal = file1.read()
    focal = float(focal)
    file1.close()
    
    red = (0, 0, 255)
    blue = (255,0,0)
    green = (0,255,0)
    
    for i in box:
        ymin, xmin, ymax, xmax = box[i][0],box[i][1],box[i][2],box[i][3]
        (left, right, top, bottom) = (int(xmin * im_width), int(xmax * im_width),
                                      int(ymin * im_height), int(ymax * im_height))
        start = (int(left),int(top))
        end = (int(right),int(bottom))
         
        

        dis = height*focal/(-top+bottom)
        if dis <=5:
            image = cv2.rectangle(image_np_with_detections, start, end, red , 3)
        elif dis >5 and dis <10:
            image = cv2.rectangle(image_np_with_detections, start, end, blue , 3)
        else:
            image = cv2.rectangle(image_np_with_detections, start, end, green , 3)
        dis = str(round(dis,2))
        image = cv2.putText(image_np_with_detections, dis, (int(left)+5,int(top)-5),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),1,cv2.LINE_AA)

    
    
#     image = cv2.resize(image, (800, 600))
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    image = cv2.putText(image, f"FPS:{fps}", (20,20), font,1, (255, 0, 0), 1, cv2.LINE_AA)


    # Display output
    img.append(image)

cap.release()

0.18700575828552246
0.1501002311706543
0.14482808113098145
0.14261865615844727
0.13811922073364258
0.1392064094543457
0.1449425220489502
0.1343519687652588
0.13949346542358398
0.14638900756835938
0.13799214363098145
0.13132977485656738
0.14134573936462402
0.1371288299560547
0.1348879337310791
0.13644862174987793
0.1350688934326172
0.1351165771484375
0.14312148094177246
0.14093899726867676
0.14477252960205078
0.14560914039611816
0.13732361793518066
0.13821887969970703
0.1476578712463379
0.145890474319458
0.1381702423095703
0.14188313484191895
0.14808177947998047
0.15041351318359375
0.1417839527130127
0.1495509147644043
0.1432056427001953
0.13698434829711914
0.1356353759765625
0.14184880256652832
0.15973687171936035
0.1386890411376953
0.14628076553344727
0.14796900749206543
0.14140629768371582
0.14775896072387695
0.13648700714111328
0.1344308853149414
0.1460099220275879
0.14029765129089355
0.14622044563293457
0.1452324390411377
0.13417720794677734
0.13999009132385254
0.13392138481140137


In [15]:
k = img[1]
height, width,layers = k.shape
size = (width,height)
out = cv2.VideoWriter('/content/data/output_inbuilt_trimnone_dist_track_out.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
 
for i in range(len(img)):
    out.write(img[i])
out.release()